In [13]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47c068c50, raw_cell="%load_ext autoreload
%reload_ext autoreload
%autor.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47f153290, execution_count=13 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47c068c50, raw_cell="%load_ext autoreload
%reload_ext autoreload
%autor.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [14]:
from pyha_analyzer import PyhaTrainer, PyhaTrainingArguments, extractors

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47c03d550, raw_cell="from pyha_analyzer import PyhaTrainer, PyhaTrainin.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47a8c52d0, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47c03d550, raw_cell="from pyha_analyzer import PyhaTrainer, PyhaTrainin.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

# Loading in Data

In [15]:
# Sorry TQ, I don't have these files mounted yet...
# peru132_extr = extractors.Peru132Extractor()
# peru_132_ads = peru132_extr("/data/XC_wav")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47a53e810, raw_cell="# Sorry TQ, I don't have these files mounted yet...." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47a53efd0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47a53e810, raw_cell="# Sorry TQ, I don't have these files mounted yet...." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [16]:
# peru_132_ads = peru_132_ads.get_provenance()

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47a8de710, raw_cell="# peru_132_ads = peru_132_ads.get_provenance()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W4sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47a8df750, execution_count=16 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47a8de710, raw_cell="# peru_132_ads = peru_132_ads.get_provenance()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W4sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [17]:
birdset_extactor = extractors.Birdset()

hsn_ads = birdset_extactor("HSN")
# per_ads = birdset_extactor("PER")

hsn_ads

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47a626250, raw_cell="birdset_extactor = extractors.Birdset()

hsn_ads =.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W5sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

DatasetDict({
    train: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 4368
    })
    valid: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 1092
    }

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47a626a50, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47a626250, raw_cell="birdset_extactor = extractors.Birdset()

hsn_ads =.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#W5sdnNjb2RlLXJlbW90ZQ%3D%3D> result=DatasetDict({
    train: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multil

BrokenPipeError: [Errno 32] Broken pipe

# Online Preprocessing example

Suppose we just wanted spectrograms with no audio preprocessing

In [18]:
hsn_ads["test"][0]["audio"]

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47c04edd0, raw_cell="hsn_ads["test"][0]["audio"]" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X10sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

{'bytes': None,
 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e7a5318118cabfab47a509edeb627860a60537535aeea20a19fced4c110d579e/HSN_001_20150708_061805_000_005.ogg'}

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47c04fbd0, execution_count=18 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc47c04edd0, raw_cell="hsn_ads["test"][0]["audio"]" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X10sdnNjb2RlLXJlbW90ZQ%3D%3D> result={'bytes': None, 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e7a5318118cabfab47a509edeb627860a60537535aeea20a19fced4c110d579e/HSN_001_20150708_061805_000_005.ogg'}>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [19]:
hsn_ads["train"][0]

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc36235c910, raw_cell="hsn_ads["train"][0]" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X11sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

{'audio': {'bytes': None,
  'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/07c8a63d645f1fba12ff7074b57f4efcee3a9f3354e0c012925d6d5e83efd6a9/XC323627.ogg'},
 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/07c8a63d645f1fba12ff7074b57f4efcee3a9f3354e0c012925d6d5e83efd6a9/XC323627.ogg',
 'start_time': None,
 'end_time': None,
 'low_freq': None,
 'high_freq': None,
 'ebird_code': 14,
 'ebird_code_multilabel': [14],
 'ebird_code_secondary': ['abetow',
  'verfly',
  'whwdov',
  'houfin',
  'moudov',
  'gamqua'],
 'call_type': 'call',
 'sex': None,
 'lat': 32.084,
 'long': -109.051,
 'length': 75,
 'microphone': 'focal',
 'license': '//creativecommons.org/licenses/by-nc-sa/4.0/',
 'source': 'xenocanto',
 'local_time': '07:38',
 'detected_events': [[0.336, 0.88],
  [2.096, 5.44],
  [4.16, 5.984],
  [5.424, 5.952],
  [9.744, 14.176],
  [13.328, 15.104],
  [20.864, 23.936],
  [23.36, 24.016],
  [24.448, 26.464],
  [25.2, 25.712],
  [27.1

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc361383910, execution_count=19 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7cc36235c910, raw_cell="hsn_ads["train"][0]" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X11sdnNjb2RlLXJlbW90ZQ%3D%3D> result={'audio': {'bytes': None, 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/07c8a63d645f1fba12ff7074b57f4efcee3a9f3354e0c012925d6d5e83efd6a9/XC323627.ogg'}, 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/07c8a63d645f1fba12ff7074b57f4efcee3a9f3354e0c012925d6d5e83efd6a9/XC323627.ogg', 'start_time': None, 'end_time': None, 'low_freq': None, 'high_freq': None, 'ebird_code': 14, 'ebird_code_multilabel': [

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
from pyha_analyzer.preprocessors import MelSpectrogramPreprocessors

# TODO: allow for normalization system

# preprocessor acts as a function for processing
# class allows us to configure parameters and whatnot
preprocessor = MelSpectrogramPreprocessors(duration=5, class_list=hsn_ads["train"].features["labels"].feature.names)

# Set split spefific transforms here
# If preprocessor had data augmentations, probably want to disable for training
hsn_ads["train"].set_transform(preprocessor)
hsn_ads["valid"].set_transform(preprocessor)
hsn_ads["test"].set_transform(preprocessor)
hsn_ads["train"][[0, 1]]["audio"][0]

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7cc47a7fe690, raw_cell="from pyha_analyzer.preprocessors import MelSpectro.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X12sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

ParameterError: hop_length=(256,) must be a positive integer

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cc362382e90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7cc47a7fe4d0, execution_count=20 error_before_exec=None error_in_exec=hop_length=(256,) must be a positive integer info=<ExecutionInfo object at 7cc47a7fe690, raw_cell="from pyha_analyzer.preprocessors import MelSpectro.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkastner-ml/home/s.perry.543/pyha-analyzer-2.0/train_demo.ipynb#X12sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
hsn_ads["train"][[0, 1]]["labels"]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0.]])

# Model Training

As a demo, implementing a model here because we haven't fleshed out the AudioDataset api yet

In the future this exists in `/pyha_analyzer/models/`

In [ ]:
# from pyha_analyzer.models.demo_CNN import ResnetConfig, ResnetModel

# # Going to make notes on anything that should be handled not here
# # This is one of these things, this should be handled by potentially pyha_trainer
# resnet50d_config = ResnetConfig(
#     num_classes=len(hsn_ads["train"].features["ebird_code"].names), input_channels=1
# )
# model = ResnetModel(resnet50d_config)
from pyha_analyzer.models import EfficentNet
model = EfficentNet(num_classes=len(hsn_ads["train"].features["ebird_code"].names))

In [ ]:
args = PyhaTrainingArguments(
    working_dir="working_dir"
)
args.num_train_epochs = 1
args.eval_steps = 20


trainer = PyhaTrainer(
    model=model,
    dataset=hsn_ads,
    training_args=args
)
trainer.train()

wandb: Currently logged in as: shperry (acoustic-species-identification) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


TrainOutput(global_step=69, training_loss=0.5874259262517038, metrics={'train_runtime': 347.4134, 'train_samples_per_second': 12.573, 'train_steps_per_second': 0.199, 'total_flos': 0.0, 'train_loss': 0.5874259262517038, 'epoch': 1.0})

In [ ]:
trainer.evaluate(eval_dataset=hsn_ads["test"], metric_key_prefix="Soundscape")

KeyboardInterrupt: 

#